In [56]:
# setting up environment
import pandas as pd
import numpy as np
import os

from imblearn.over_sampling import SMOTENC

from dotenv import load_dotenv

load_dotenv()

True

In [57]:
# import dataset

train_df = pd.read_csv(os.getcwd()+'/datasets/training.csv')
train_df.head()

,id,age,sex,is_smoking,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,989,42,0,1,3.0,0,0,0,0,227.786207,129.0,81.0,25.190207,75.344828,76.759398,0
1,690,60,1,1,30.0,0,0,0,0,256.610526,122.5,68.5,26.063895,76.052632,82.284091,0
2,3054,42,0,1,9.0,0,0,0,0,227.786207,126.0,81.0,25.190207,75.344828,76.759398,0
3,793,54,0,1,5.0,0,0,1,0,246.028302,139.0,75.0,26.125701,76.364486,83.030303,0
4,138,53,0,0,0.0,0,0,1,0,248.359649,172.5,91.0,26.014298,77.754386,80.441176,0


In [58]:
# drop colomn id before sampling
def dropColomn(df, col):
    try:
        df = df.drop(col,1)
    except:
        print('colomn {0} not found'.format(col))
    return df

train_df = dropColomn(train_df,'id')
train_df.head()

,age,sex,is_smoking,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,42,0,1,3.0,0,0,0,0,227.786207,129.0,81.0,25.190207,75.344828,76.759398,0
1,60,1,1,30.0,0,0,0,0,256.610526,122.5,68.5,26.063895,76.052632,82.284091,0
2,42,0,1,9.0,0,0,0,0,227.786207,126.0,81.0,25.190207,75.344828,76.759398,0
3,54,0,1,5.0,0,0,1,0,246.028302,139.0,75.0,26.125701,76.364486,83.030303,0
4,53,0,0,0.0,0,0,1,0,248.359649,172.5,91.0,26.014298,77.754386,80.441176,0


In [59]:
# original data

print('shape of original data: {0}'.format(train_df.shape))
train_df.groupby('TenYearCHD').size()

shape of original data: (2170, 15)


TenYearCHD
0    1841
1     329
dtype: int64

In [60]:
# split features and labels

labels = np.array(train_df.pop('TenYearCHD'))
features = np.array(train_df)

In [61]:
# splited shape
print('shape of original features : {0}'.format(features.shape))
print('shape of original labels : {0}'.format(labels.shape))

shape of original features : (2170, 14)
shape of original labels : (2170,)


In [62]:
# run sampling using SMOTE-ENC
CATEGORICAL_FEATURES = [1,2,4,5,6,7]

sm = SMOTENC(random_state=42, categorical_features=CATEGORICAL_FEATURES)
features_res, labels_res = sm.fit_resample(features, labels)

#smote result
print('shape of sampled features : {0}'.format(features_res.shape))
print('shape of sampled labels : {0}'.format(labels_res.shape))

shape of sampled features : (3682, 14)
shape of sampled labels : (3682,)


In [63]:
# expands labels dimension to merge data 
labels_res_new = np.expand_dims(labels_res, axis=0)

# merge sampled features and labels 
balance_df = pd.DataFrame(np.concatenate((features_res, labels_res_new.T), axis=1),columns=[
    'age', 'sex',	'is_smoking',	'cigsPerDay',	
    'BPMeds',	'prevalentStroke',	'prevalentHyp',	
    'diabetes',	'totChol',	'sysBP',	'diaBP',	
    'BMI',	'heartRate',	'glucose',	'TenYearCHD']
    )

balance_df.head()

,age,sex,is_smoking,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,42.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,227.786207,129.0,81.0,25.190207,75.344828,76.759398,0.0
1,60.0,1.0,1.0,30.0,0.0,0.0,0.0,0.0,256.610526,122.5,68.5,26.063895,76.052632,82.284091,0.0
2,42.0,0.0,1.0,9.0,0.0,0.0,0.0,0.0,227.786207,126.0,81.0,25.190207,75.344828,76.759398,0.0
3,54.0,0.0,1.0,5.0,0.0,0.0,1.0,0.0,246.028302,139.0,75.0,26.125701,76.364486,83.030303,0.0
4,53.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,248.359649,172.5,91.0,26.014298,77.754386,80.441176,0.0


In [64]:
# preprocessing after sampling

# drop duplicated colomn
balance_df = balance_df.drop_duplicates()
print('shape of sampled training dataset after drop duplicates : {0}'.format(balance_df.shape))

unWantedSampledData = os.getenv('NUMOFUNWANTEDSAMPLEDDATA')
if unWantedSampledData != '':
    try:
        unWantedSampledDataInt = int(unWantedSampledData)
        balance_df.drop(balance_df.tail(unWantedSampledDataInt).index,inplace=True)
    except:
        print('cant process drop unwanted data')
    print('shape of sampled training dataset after drop unwanted data : {0}'.format(balance_df.shape))


shape of sampled training dataset after drop duplicates : (3676, 15)
shape of sampled training dataset after drop unwanted data : (3176, 15)


In [65]:
# shuffle sampled dataset n times
try:
    NUMBEROFSHUFFLING = int(os.getenv('NUMOFSHUFFLINGDATA'))

    for i in range(NUMBEROFSHUFFLING):
        balance_df = balance_df.sample(frac=1, random_state=i*16)
        print('shuffle sampeld dataset {0} time'.format(i))
except:
    print('failed to shuffle sampled dataset')

shuffle sampeld dataset 0 time
shuffle sampeld dataset 1 time
shuffle sampeld dataset 2 time
shuffle sampeld dataset 3 time
shuffle sampeld dataset 4 time
shuffle sampeld dataset 5 time
shuffle sampeld dataset 6 time
shuffle sampeld dataset 7 time
shuffle sampeld dataset 8 time
shuffle sampeld dataset 9 time
shuffle sampeld dataset 10 time
shuffle sampeld dataset 11 time
shuffle sampeld dataset 12 time
shuffle sampeld dataset 13 time
shuffle sampeld dataset 14 time
shuffle sampeld dataset 15 time


In [66]:
balance_df.to_csv(os.getcwd()+'/datasets/sampled_dataset.csv',  index_label='id')